# 📊 Building the Medallion Architecture
*Create a complete F1 analytics pipeline using the data lake design pattern*

---

## 🎯 What We'll Build

In this notebook, we'll build a complete data pipeline following the medallion architecture pattern:

1. **Bronze Layer**: Raw data ingestion from Volume
2. **Silver Layer**: Clean and transform the data
3. **Gold Layer**: Create analytics-ready tables

We'll use the F1 dataset that we prepared in the setup notebook to analyze Formula 1 racing history.

**Let's get started!** 🚀

## 📋 Prerequisites

Before running this notebook, make sure you've completed:
1. The `00_Setup_Data.ipynb` notebook to download F1 data into the Volume
2. Attached this notebook to a cluster with Spark 3.3+

Let's first verify that our data is available in the Volume:

In [ ]:
# Check that our Volume exists and contains data
import os

volume_path = "/Volumes/main/default/f1_raw_data/"

try:
    files = os.listdir(volume_path)
    print(f"✅ Volume found with {len(files)} files:")
    
    for file in files:
        if file.endswith('.csv'):
            file_path = os.path.join(volume_path, file)
            file_size = os.path.getsize(file_path)
            print(f"  - {file}: {file_size:,} bytes")
            
except Exception as e:
    print(f"❌ Error accessing volume: {str(e)}")
    print("Please run the 00_Setup_Data.ipynb notebook first to prepare the data.")
    raise Exception("Data volume not found or not accessible. Run setup notebook first.")

## 1️⃣ Bronze Layer: Raw Data Ingestion

The Bronze layer represents the raw data ingested from source systems with minimal transformation.
We'll create tables from the CSV files in our Volume using the COPY INTO command.

Let's start by creating tables for races, drivers, and results:

In [ ]:
-- First, let's make sure we don't have any existing tables
DROP TABLE IF EXISTS main.default.bronze_races;
DROP TABLE IF EXISTS main.default.bronze_drivers;
DROP TABLE IF EXISTS main.default.bronze_results;

-- Create bronze_races table
CREATE TABLE IF NOT EXISTS main.default.bronze_races (
  raceId INT,
  year INT,
  round INT,
  circuitId INT,
  name STRING,
  date DATE,
  time STRING,
  url STRING
) USING DELTA;

-- Create bronze_drivers table
CREATE TABLE IF NOT EXISTS main.default.bronze_drivers (
  driverId INT,
  driverRef STRING,
  number INT,
  code STRING,
  forename STRING,
  surname STRING,
  dob DATE,
  nationality STRING,
  url STRING
) USING DELTA;

-- Create bronze_results table
CREATE TABLE IF NOT EXISTS main.default.bronze_results (
  resultId INT,
  raceId INT,
  driverId INT,
  constructorId INT,
  number INT,
  grid INT,
  position INT,
  positionText STRING,
  positionOrder INT,
  points FLOAT,
  laps INT,
  time STRING,
  milliseconds INT,
  fastestLap INT,
  rank INT,
  fastestLapTime STRING,
  fastestLapSpeed FLOAT,
  statusId INT
) USING DELTA;

In [ ]:
-- Load data into bronze_races using COPY INTO
COPY INTO main.default.bronze_races
FROM '/Volumes/main/default/f1_raw_data/races.csv'
FILEFORMAT = CSV
FORMAT_OPTIONS ('header' = 'true', 'inferSchema' = 'true')
COPY_OPTIONS ('mergeSchema' = 'true');

-- Check that data was loaded
SELECT 
  'bronze_races' as table_name,
  COUNT(*) as record_count,
  MIN(year) as earliest_year,
  MAX(year) as latest_year
FROM main.default.bronze_races;

In [ ]:
-- Load data into bronze_drivers using COPY INTO
COPY INTO main.default.bronze_drivers
FROM '/Volumes/main/default/f1_raw_data/drivers.csv'
FILEFORMAT = CSV
FORMAT_OPTIONS ('header' = 'true', 'inferSchema' = 'true')
COPY_OPTIONS ('mergeSchema' = 'true');

-- Check that data was loaded
SELECT 
  'bronze_drivers' as table_name,
  COUNT(*) as record_count,
  COUNT(DISTINCT nationality) as unique_nationalities,
  MIN(dob) as earliest_dob,
  MAX(dob) as latest_dob
FROM main.default.bronze_drivers;

In [ ]:
-- Load data into bronze_results using COPY INTO
COPY INTO main.default.bronze_results
FROM '/Volumes/main/default/f1_raw_data/results.csv'
FILEFORMAT = CSV
FORMAT_OPTIONS ('header' = 'true', 'inferSchema' = 'true')
COPY_OPTIONS ('mergeSchema' = 'true');

-- Check that data was loaded
SELECT 
  'bronze_results' as table_name,
  COUNT(*) as record_count,
  COUNT(DISTINCT raceId) as unique_races,
  COUNT(DISTINCT driverId) as unique_drivers,
  SUM(points) as total_points
FROM main.default.bronze_results;

## 2️⃣ Silver Layer: Clean and Transform

The Silver layer contains cleaner, more refined data. Here we'll:
- Fix data types
- Add full names for drivers
- Standardize date/time formats
- Remove/handle nulls and errors
- Apply consistent naming conventions

In [ ]:
-- First, drop any existing silver tables
DROP TABLE IF EXISTS main.default.silver_races;
DROP TABLE IF EXISTS main.default.silver_drivers;
DROP TABLE IF EXISTS main.default.silver_results;

-- Create silver_races with improved structure and data quality
CREATE TABLE main.default.silver_races AS
SELECT
  raceId,
  year,
  round,
  circuitId,
  name as race_name,
  -- Properly format date and handle nulls
  date,
  CASE 
    WHEN time IS NULL THEN '00:00:00' 
    ELSE time 
  END as start_time,
  -- Extract and add useful derived columns
  year as season,
  CASE
    WHEN year < 1970 THEN 'Early F1 Era'
    WHEN year < 1980 THEN '70s Era'
    WHEN year < 1990 THEN '80s Era'
    WHEN year < 2000 THEN '90s Era'
    WHEN year < 2010 THEN '2000s Era'
    WHEN year < 2014 THEN 'Early 2010s'
    WHEN year < 2022 THEN 'Hybrid Era'
    ELSE 'Current Era'
  END as f1_era,
  -- Add metadata
  current_timestamp() as processed_at
FROM main.default.bronze_races
WHERE year IS NOT NULL  -- Data quality: exclude rows with null year
ORDER BY year, round;

-- Check the results
SELECT * FROM main.default.silver_races LIMIT 10;

In [ ]:
-- Create silver_drivers with cleaner structure
CREATE TABLE main.default.silver_drivers AS
SELECT
  driverId,
  driverRef as driver_ref,
  -- Handle nulls in number and code fields
  CAST(number AS INT) as driver_number,
  code as driver_code,
  -- Create full name
  forename as first_name,
  surname as last_name,
  CONCAT(forename, ' ', surname) as full_name,
  -- Clean date format
  dob as date_of_birth,
  -- Calculate age (simplified)
  CAST(year(current_date()) - year(dob) AS INT) as current_age,
  nationality,
  -- Add metadata
  current_timestamp() as processed_at
FROM main.default.bronze_drivers
-- Ensure we have at least a name
WHERE forename IS NOT NULL AND surname IS NOT NULL;

-- Check the results
SELECT * FROM main.default.silver_drivers LIMIT 10;

In [ ]:
-- Create silver_results with clean, consistent structure
CREATE TABLE main.default.silver_results AS
SELECT
  resultId as result_id,
  raceId as race_id,
  driverId as driver_id,
  constructorId as constructor_id,
  -- Handle nulls in numeric columns
  CAST(COALESCE(number, -1) AS INT) as car_number,
  CAST(grid AS INT) as starting_grid,
  -- Clean position data
  CASE 
    WHEN position = 'N' THEN NULL
    WHEN position = 'R' THEN NULL
    WHEN position = 'D' THEN NULL
    ELSE CAST(position AS INT)
  END as finish_position,
  positionText as position_text,
  positionOrder as position_order,
  points,
  laps,
  -- Format time properly
  time as race_time,
  milliseconds as race_time_ms,
  -- FastestLap info
  CAST(COALESCE(fastestLap, -1) AS INT) as fastest_lap,
  CAST(COALESCE(rank, -1) AS INT) as fastest_lap_rank,
  fastestLapTime as fastest_lap_time,
  CAST(COALESCE(fastestLapSpeed, -1) AS FLOAT) as fastest_lap_speed,
  statusId as status_id,
  -- Add metadata
  current_timestamp() as processed_at
FROM main.default.bronze_results;

-- Check the results
SELECT * FROM main.default.silver_results LIMIT 10;

## 3️⃣ Gold Layer: Analytics-Ready Tables

The Gold layer contains business-level aggregations ready for analytics. 
Let's create tables that answer key questions about F1 racing.

In [ ]:
-- First, drop any existing gold tables
DROP TABLE IF EXISTS main.default.gold_driver_standings;
DROP TABLE IF EXISTS main.default.gold_season_stats;

-- Create gold_driver_standings
-- This will contain career statistics for all drivers
CREATE TABLE main.default.gold_driver_standings AS
SELECT 
  d.driver_id,
  d.full_name,
  d.nationality,
  d.date_of_birth,
  -- Career stats
  COUNT(DISTINCT r.race_id) as total_races,
  -- Count wins (position = 1)
  SUM(CASE WHEN r.finish_position = 1 THEN 1 ELSE 0 END) as wins,
  -- Count podiums (position <= 3)
  SUM(CASE WHEN r.finish_position <= 3 THEN 1 ELSE 0 END) as podiums,
  -- Total career points
  SUM(r.points) as total_career_points,
  -- Average points per race
  ROUND(SUM(r.points) / COUNT(DISTINCT r.race_id), 2) as points_per_race,
  -- Win percentage
  ROUND(SUM(CASE WHEN r.finish_position = 1 THEN 1 ELSE 0 END) * 100.0 / 
        COUNT(DISTINCT r.race_id), 2) as win_percentage,
  -- Career span
  MIN(race.season) as career_start_year,
  MAX(race.season) as career_end_year,
  MAX(race.season) - MIN(race.season) + 1 as career_span_years,
  -- Metadata
  current_timestamp() as processed_at
FROM main.default.silver_drivers d
JOIN main.default.silver_results r ON d.driver_id = r.driver_id
JOIN main.default.silver_races race ON r.race_id = race.raceId
GROUP BY 
  d.driver_id,
  d.full_name,
  d.nationality,
  d.date_of_birth
-- Only include drivers with at least one race
HAVING COUNT(DISTINCT r.race_id) >= 1
ORDER BY total_career_points DESC;

-- Check the results
SELECT * FROM main.default.gold_driver_standings LIMIT 10;

In [ ]:
-- Create gold_season_stats
-- This will contain statistics for each F1 season
CREATE TABLE main.default.gold_season_stats AS
WITH race_counts AS (
  SELECT 
    season,
    f1_era,
    COUNT(*) as total_races
  FROM main.default.silver_races
  GROUP BY season, f1_era
),
season_drivers AS (
  SELECT
    race.season,
    COUNT(DISTINCT r.driver_id) as unique_drivers,
    COUNT(DISTINCT r.constructor_id) as unique_constructors
  FROM main.default.silver_results r
  JOIN main.default.silver_races race ON r.race_id = race.raceId
  GROUP BY race.season
),
race_winners AS (
  SELECT
    race.season,
    COUNT(DISTINCT r.driver_id) as unique_race_winners,
    COUNT(DISTINCT r.constructor_id) as unique_winning_constructors
  FROM main.default.silver_results r
  JOIN main.default.silver_races race ON r.race_id = race.raceId
  WHERE r.finish_position = 1
  GROUP BY race.season
),
race_completion AS (
  SELECT
    race.season,
    AVG(CASE WHEN r.finish_position IS NOT NULL THEN 1.0 ELSE 0.0 END) as completion_rate
  FROM main.default.silver_results r
  JOIN main.default.silver_races race ON r.race_id = race.raceId
  GROUP BY race.season
),
season_points AS (
  SELECT
    race.season,
    SUM(r.points) as total_points_awarded
  FROM main.default.silver_results r
  JOIN main.default.silver_races race ON r.race_id = race.raceId
  GROUP BY race.season
)
SELECT
  rc.season,
  rc.f1_era,
  rc.total_races,
  sd.unique_drivers,
  sd.unique_constructors,
  COALESCE(rw.unique_race_winners, 0) as unique_race_winners,
  COALESCE(rw.unique_winning_constructors, 0) as unique_winning_constructors,
  ROUND(rcp.completion_rate, 3) as completion_rate,
  COALESCE(sp.total_points_awarded, 0) as total_points_awarded,
  current_timestamp() as processed_at
FROM race_counts rc
LEFT JOIN season_drivers sd ON rc.season = sd.season
LEFT JOIN race_winners rw ON rc.season = rw.season
LEFT JOIN race_completion rcp ON rc.season = rcp.season
LEFT JOIN season_points sp ON rc.season = sp.season
ORDER BY rc.season;

-- Check the results
SELECT * FROM main.default.gold_season_stats ORDER BY season DESC LIMIT 10;

## 4️⃣ Verify the Medallion Architecture

Let's visualize our complete medallion architecture by checking record counts at each layer:

In [ ]:
# Create a dataframe showing table counts at each layer
from pyspark.sql.functions import col, lit

# Function to get table record count
def get_table_count(table_name):
    return spark.sql(f"SELECT COUNT(*) as count FROM main.default.{table_name}").collect()[0].count

# Collect counts for each layer
bronze_counts = {
    'bronze_races': get_table_count('bronze_races'),
    'bronze_drivers': get_table_count('bronze_drivers'),
    'bronze_results': get_table_count('bronze_results')
}

silver_counts = {
    'silver_races': get_table_count('silver_races'),
    'silver_drivers': get_table_count('silver_drivers'),
    'silver_results': get_table_count('silver_results')
}

gold_counts = {
    'gold_driver_standings': get_table_count('gold_driver_standings'),
    'gold_season_stats': get_table_count('gold_season_stats')
}

# Create dataframe for visualization
data = []

# Add bronze layer
for table, count in bronze_counts.items():
    data.append(("Bronze", table, count))
    
# Add silver layer
for table, count in silver_counts.items():
    data.append(("Silver", table, count))
    
# Add gold layer
for table, count in gold_counts.items():
    data.append(("Gold", table, count))

# Create dataframe
medallion_df = spark.createDataFrame(data, ["layer", "table", "record_count"])

# Display the results
display(medallion_df.orderBy("layer", "table"))

# Calculate totals by layer
layer_totals = medallion_df.groupBy("layer").sum("record_count").withColumnRenamed("sum(record_count)", "total_records")
display(layer_totals.orderBy("layer"))

## 5️⃣ Explore the Data Model

Let's see some example queries that use our medallion architecture:

In [ ]:
-- Top 10 drivers of all time by wins
SELECT 
  full_name,
  nationality,
  wins,
  podiums,
  total_races,
  ROUND(win_percentage, 1) as win_pct,
  total_career_points
FROM main.default.gold_driver_standings
ORDER BY wins DESC
LIMIT 10;

In [ ]:
-- Most competitive seasons (highest number of different winners)
SELECT 
  season,
  f1_era,
  total_races,
  unique_race_winners,
  unique_winning_constructors,
  ROUND(unique_race_winners * 100.0 / total_races, 1) as winner_diversity_pct
FROM main.default.gold_season_stats
WHERE total_races >= 10
ORDER BY unique_race_winners DESC
LIMIT 10;

In [ ]:
-- Countries with most F1 driver success
SELECT
  d.nationality,
  COUNT(DISTINCT d.driver_id) as total_drivers,
  SUM(d.wins) as total_wins,
  SUM(d.podiums) as total_podiums,
  ROUND(SUM(d.total_career_points), 0) as total_points,
  MAX(d.full_name) as most_successful_driver,
  MAX(d.wins) as most_wins_by_driver
FROM main.default.gold_driver_standings d
GROUP BY d.nationality
HAVING SUM(d.wins) > 0
ORDER BY total_wins DESC
LIMIT 10;

## ✅ Medallion Architecture Complete!

Congratulations! You've successfully:
- ✅ Created a Bronze layer with raw F1 data
- ✅ Built a Silver layer with cleaned and transformed data
- ✅ Developed a Gold layer with analytics-ready tables
- ✅ Verified and explored the complete medallion architecture

### 📈 Your F1 Data Pipeline Architecture:

```
Volume: /Volumes/main/default/f1_raw_data/
  ↓ COPY INTO
Bronze Layer:
  bronze_races, bronze_drivers, bronze_results
  ↓ Clean & Transform
Silver Layer:
  silver_races, silver_drivers, silver_results
  ↓ Aggregate & Model
Gold Layer:
  gold_driver_standings, gold_season_stats
  ↓ Analytics & Dashboards
```

**Next Steps:**
- Continue to [03_Unity_Catalog_Demo.ipynb](03_Unity_Catalog_Demo.ipynb) to learn about data governance and lineage
- Create dashboards using the gold tables
- Set up automated refresh jobs for this pipeline